# Big Contest 데이처 전처리

## 1. 데이터 불러오기

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
from datetime import datetime

- 제공데이터에는 **실적데이터**와 **시청률데이터**가 있다.
<br>
<br>
- **실적데이터**를 먼저 불러오자.
    - **실적데이터**의 **한글** 칼럼명을 **영문**으로 **변경**하자.
        - 칼럼을 통해 데이터를 조회할 때 한/영 키를 눌러줘야 하는 복잡함을 해소!! 

In [2]:
df_excel = pd.read_excel('C:/Users/Playdata/10. side_project/04. Big Contest/2020빅콘테스트 문제데이터(데이터분석분야-챔피언리그)/01_제공데이터/2020 빅콘테스트 데이터분석분야-챔피언리그_2019년 실적데이터.xlsx',
                         header=1)
                         
df_excel.rename(columns = {'방송일시':'date', '노출(분)':'exp_mins', '마더코드':'mom_code', '상품코드':'pd_code',
                 '상품명':'pd_name', '상품군':'pd_group', '판매단가':'pd_price', '취급액':'sales'}, inplace=True)
df_excel

,date,exp_mins,mom_code,pd_code,pd_name,pd_group,pd_price,sales
0,2019-01-01 06:00:00,20.000000,100346,201072,테이트 남성 셀린니트3종,의류,39900,2099000.0
1,2019-01-01 06:00:00,NaN,100346,201079,테이트 여성 셀린니트3종,의류,39900,4371000.0
2,2019-01-01 06:20:00,20.000000,100346,201072,테이트 남성 셀린니트3종,의류,39900,3262000.0
3,2019-01-01 06:20:00,NaN,100346,201079,테이트 여성 셀린니트3종,의류,39900,6955000.0
4,2019-01-01 06:40:00,20.000000,100346,201072,테이트 남성 셀린니트3종,의류,39900,6672000.0
...,...,...,...,...,...,...,...,...
38304,2020-01-01 00:20:00,20.000000,100073,200196,삼성화재 행복한파트너 주택화재보험(1912),무형,0,NaN
38305,2020-01-01 00:40:00,20.000000,100073,200196,삼성화재 행복한파트너 주택화재보험(1912),무형,0,NaN
38306,2020-01-01 01:00:00,20.000000,100073,200196,삼성화재 행복한파트너 주택화재보험(1912),무형,0,NaN
38307,2020-01-01 01:20:00,20.000000,100490,201478,더케이 예다함 상조서비스(티포트),무형,0,NaN


## 2. 데이터 살펴보기

- 우선 **Null(결측치)**가 있는지 확인해보자.

In [3]:
df_excel.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38309 entries, 0 to 38308
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   date      38309 non-null  datetime64[ns]
 1   exp_mins  21525 non-null  float64       
 2   mom_code  38309 non-null  int64         
 3   pd_code   38309 non-null  int64         
 4   pd_name   38309 non-null  object        
 5   pd_group  38309 non-null  object        
 6   pd_price  38309 non-null  int64         
 7   sales     37372 non-null  float64       
dtypes: datetime64[ns](1), float64(2), int64(3), object(2)
memory usage: 2.3+ MB


- **exp_mins(노출(분))**와 **sales(취급액)**에 **Null**값이 존재한다.
    - sales(취급액)의 Null값이 상대적으로 적은 수이기 때문에 먼저 확인해주자.

In [4]:
intangible = df_excel[(df_excel['pd_group'] == '무형')].index.tolist()
len(intangible)

937

- **pd_group(상품군)**이 **무형**인 상품의 index를 리스트에 넣어주고 카운팅해보니 ***937***개가 존재한다.

In [5]:
seles_null = df_excel[df_excel['sales'].isnull()].index.tolist()
len(seles_null)

937

- **sales(취급액)**이 **Null**값인 상품의 index를 리스트에 넣어주고 카운팅해보니 역시 ***937***개가 존재한다.
    - 두 상품이 같은 index를 가진다면 sales(취급앱)이 Null인 상품은 모두 무형의 pd_group(상품군)을 가지고 있다고 결론 내릴 수 있다.

In [6]:
compare = []
for i, j in zip(intangible, seles_null):
    if i != j:
        compare.append(i)
len(compare)

0

- sales(취급액)이 Null인 모든 상품의 pd_group(상품군)은 무형이다.
    - 아울러 pd_price(판매단가) 역시 0이다.

In [7]:
price_zero = df_excel[df_excel['pd_price'] == 0].index.tolist()
len(price_zero)

937

In [8]:
compare = []
for i, j in zip(intangible, price_zero):
    if i != j:
        compare.append(i)
len(compare)

0

- 참고로 pd_price(판매단가)에는 Null값이 존재하지 않는다.

In [9]:
price_null = df_excel[df_excel['pd_price'].isnull()].index.tolist()
len(price_null)

0

- pd_group(상품군)이 무형인 상품을 파악해보자.

In [10]:
df_intangible = df_excel[(df_excel['pd_group'] == '무형') & (df_excel['sales'].isnull()) & (df_excel['pd_price'] == 0)]
df_intangible

,date,exp_mins,mom_code,pd_code,pd_name,pd_group,pd_price,sales
2318,2019-01-25 19:00:00,60.000000,100507,201567,TCOM_처브치아보험,무형,0,NaN
2845,2019-01-31 18:00:00,60.000000,100507,201567,TCOM_처브치아보험,무형,0,NaN
3332,2019-02-05 18:00:00,60.000000,100507,201567,TCOM_처브치아보험,무형,0,NaN
6570,2019-03-09 00:00:00,20.000000,100324,201095,온라인투어 동남아 나트랑/빈펄랜드 (190309),무형,0,NaN
6571,2019-03-09 00:20:00,20.000000,100324,201095,온라인투어 동남아 나트랑/빈펄랜드 (190309),무형,0,NaN
...,...,...,...,...,...,...,...,...
38304,2020-01-01 00:20:00,20.000000,100073,200196,삼성화재 행복한파트너 주택화재보험(1912),무형,0,NaN
38305,2020-01-01 00:40:00,20.000000,100073,200196,삼성화재 행복한파트너 주택화재보험(1912),무형,0,NaN
38306,2020-01-01 01:00:00,20.000000,100073,200196,삼성화재 행복한파트너 주택화재보험(1912),무형,0,NaN
38307,2020-01-01 01:20:00,20.000000,100490,201478,더케이 예다함 상조서비스(티포트),무형,0,NaN


- 아래와 같이 모두 비슷한 상품으로 분류할 수 있다.
    - 보험, 여행, 상조, 렌탈서비스 (렌터카, 영어, 의료기기 등), 휴대폰 상담 등

In [11]:
product_names = set(df_intangible['pd_name'].tolist())
product_names

{' 용인공원라이프 스마트라이프48',
 '(렌탈) 헬스리아 반신욕기',
 '(무)메리츠 올바른 간편보험1810',
 'DB통큰간편암플랜',
 'SKT휴대폰 (삼성갤럭시 노트9)',
 'SK매직정수기 3종',
 'TCOM_X-RUN Serise',
 'TCOM_관절염 치료 의료기기 닥터88',
 'TCOM_라클라우드 모션배드',
 'TCOM_바디프랜드 아제라',
 'TCOM_아이러너 런닝머신 렌탈',
 'TCOM_아이러너 런닝머신 렌탈(x1,x3)',
 'TCOM_처브치아보험',
 'TCOM_현대해상 (무)하이콜운전자상해보험',
 '[상담접수상품]뇌새김 영어 렌탈',
 '넥센타이어 렌탈서비스',
 '넥센타이어 렌탈서비스(특약)',
 '대명아임레디 더행복한플랜(싱글)',
 '더케이 예다함 상조서비스(티포트)',
 '롯데JTB 장가계(190726)',
 '롯데JTB 장가계(190803)',
 '롯데JTB 청도(190928)상담예약',
 '롯데JTB 태항산5일(190628)',
 '롯데관광 북경(191213)상담예약',
 '롯데관광 세부 (191227)상담예약',
 '롯데관광 시안(191101)상담예약',
 '롯데관광 인도 네팔(191115)상담예약',
 '롯데관광 호주(191025)상담예약',
 '롯데렌터카',
 '삼성화재 천만안심 건강보험(암보험, 1907)',
 '삼성화재 행복한파트너 주택화재보험(1912)',
 '실속하나로암보험 체증형',
 '여행박사 대만(190503)',
 '온라인투어 대련 아시아나(191108) 상담예약',
 '온라인투어 대련(191025) 상담예약',
 '온라인투어 동경/오다이바/하코네 (0601)',
 '온라인투어 동남아 나트랑/빈펄랜드 (190309)',
 '온라인투어 동남아 다낭5일(0427)',
 '온라인투어 미동부/캐나다 (190608)',
 '온라인투어 북경 (191227) 상담예약',
 '온라인투어 북경(190706)',
 '온라인투어 북해도(190511)',
 '온라인투어 오사카(0406)',
 '온라인투어 오키나와 (1

- 예측에는 필요하지 않은 정보들이니 제거해주자.

In [12]:
df_excel_drop = df_excel.drop(index=df_excel[(df_excel['pd_group'] == '무형') & (df_excel['sales'].isnull())].index)
df_excel_drop

,date,exp_mins,mom_code,pd_code,pd_name,pd_group,pd_price,sales
0,2019-01-01 06:00:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,2099000.0
1,2019-01-01 06:00:00,NaN,100346,201079,테이트 여성 셀린니트3종,의류,39900,4371000.0
2,2019-01-01 06:20:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,3262000.0
3,2019-01-01 06:20:00,NaN,100346,201079,테이트 여성 셀린니트3종,의류,39900,6955000.0
4,2019-01-01 06:40:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,6672000.0
...,...,...,...,...,...,...,...,...
38299,2019-12-31 23:40:00,NaN,100448,201391,일시불쿠첸압력밥솥 6인용,주방,148000,10157000.0
38300,2020-01-01 00:00:00,20.0,100448,201383,무이자쿠첸압력밥솥 10인용,주방,178000,50929000.0
38301,2020-01-01 00:00:00,NaN,100448,201390,일시불쿠첸압력밥솥 10인용,주방,168000,104392000.0
38302,2020-01-01 00:00:00,NaN,100448,201384,무이자쿠첸압력밥솥 6인용,주방,158000,13765000.0


- 비교 결과 성공적으로 제외하였다.

In [13]:
len(df_excel) - len(df_excel_drop)

937

- 이어서 exp_mins(노출(분))의 null값을 살펴보자.

In [14]:
df_exposed_null = df_excel_drop[df_excel_drop['exp_mins'].isnull()]
df_exposed_null.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16784 entries, 1 to 38303
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   date      16784 non-null  datetime64[ns]
 1   exp_mins  0 non-null      float64       
 2   mom_code  16784 non-null  int64         
 3   pd_code   16784 non-null  int64         
 4   pd_name   16784 non-null  object        
 5   pd_group  16784 non-null  object        
 6   pd_price  16784 non-null  int64         
 7   sales     16784 non-null  float64       
dtypes: datetime64[ns](1), float64(2), int64(3), object(2)
memory usage: 1.2+ MB


- 총 16784개로 전체 데이터 셋의 45%에 해당한다.
    - 또한 모두 pd_price(판매단가)와 sales(취급액)을 가지고 있기 때문에 예측에 필요한 정보이므로 제거해줄 수 없다.

In [15]:
len(df_exposed_null)/len(df_excel_drop)*100

44.91062827785508

- 적절한 값으로 대체할 수 있는지 찾아봐야 한다.
<br>
<br>
- 우선 동일한 mom_code(마더코드)를 가진 다른 값들이 존재하는지 알아보자.
    1. df_exposed의 mom_code(마더코드)를 리스트에 담는다.
    2. df_excel_drop에서 동일한 mom_code(마더코드)를 가진 상품을 찾아본다.
    3. 동일한 mom_code(마더코드)를 가진 상품을 모두 호출한다.
    4. 어떤 관계가 있는지 찾아본다.

In [16]:
df_exposed_null_mc = list(set(df_exposed_null['mom_code'].tolist()))
len(df_exposed_null_mc)

285

- 총 285개의 Unique한 mom_code(마더코드)가 존재한다.
<br>
<br>
- 이제 전제 데이터셋에서 동일한 mom_code(마더코드)를 가진 모든 상품을 호출해보자.

In [17]:
df_same_mom_code = df_excel_drop[df_excel_drop['mom_code'].isin(df_exposed_null_mc)]
df_same_mom_code.head(30)

,date,exp_mins,mom_code,pd_code,pd_name,pd_group,pd_price,sales
0,2019-01-01 06:00:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,2099000.0
1,2019-01-01 06:00:00,NaN,100346,201079,테이트 여성 셀린니트3종,의류,39900,4371000.0
2,2019-01-01 06:20:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,3262000.0
3,2019-01-01 06:20:00,NaN,100346,201079,테이트 여성 셀린니트3종,의류,39900,6955000.0
4,2019-01-01 06:40:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,6672000.0
5,2019-01-01 06:40:00,NaN,100346,201079,테이트 여성 셀린니트3종,의류,39900,9337000.0
25,2019-01-01 14:00:00,30.0,100377,201220,그렉노먼 남성 구스다운 롱 벤치코트,의류,119000,11269000.0
26,2019-01-01 14:00:00,NaN,100377,201226,그렉노먼 여성 구스다운 롱 벤치코트,의류,119000,20841000.0
27,2019-01-01 14:30:00,30.0,100377,201220,그렉노먼 남성 구스다운 롱 벤치코트,의류,119000,40526000.0
28,2019-01-01 14:30:00,NaN,100377,201226,그렉노먼 여성 구스다운 롱 벤치코트,의류,119000,47294000.0


- exp_mins(노출(분))의 Null값은 같은 시간대에 같은 상품을 팔 때 발생하는데 기준은 아래와 같다.
    - 다만, 성별 / 사용용도(6인용 vs 10인용) / 결제방법 (무이자 / 일시불) 등 상품을 세분화하여 등록했는지에 따라서 null값이 발생했다.
<br>
<br>
- 같은 시간대 같은 상품의 노출(분)을 사용하여 Null값을 채워주자.
    - fillna의 메소드 중 ffill을 이용하여 Null을 채워주자.
    - ffill 은 Null값의 바로 전 값을 사용해 Null을 채운다.

In [18]:
df_mom_code_ffill = df_excel_drop.fillna(method = 'ffill')
df_mom_code_ffill.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 37372 entries, 0 to 38303
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   date      37372 non-null  datetime64[ns]
 1   exp_mins  37372 non-null  float64       
 2   mom_code  37372 non-null  int64         
 3   pd_code   37372 non-null  int64         
 4   pd_name   37372 non-null  object        
 5   pd_group  37372 non-null  object        
 6   pd_price  37372 non-null  int64         
 7   sales     37372 non-null  float64       
dtypes: datetime64[ns](1), float64(2), int64(3), object(2)
memory usage: 2.6+ MB


In [19]:
df_mom_code_ffill.head(30)

,date,exp_mins,mom_code,pd_code,pd_name,pd_group,pd_price,sales
0,2019-01-01 06:00:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,2099000.0
1,2019-01-01 06:00:00,20.0,100346,201079,테이트 여성 셀린니트3종,의류,39900,4371000.0
2,2019-01-01 06:20:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,3262000.0
3,2019-01-01 06:20:00,20.0,100346,201079,테이트 여성 셀린니트3종,의류,39900,6955000.0
4,2019-01-01 06:40:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,6672000.0
5,2019-01-01 06:40:00,20.0,100346,201079,테이트 여성 셀린니트3종,의류,39900,9337000.0
6,2019-01-01 07:00:00,20.0,100305,200974,오모떼 레이스 파운데이션 브라,속옷,59000,6819000.0
7,2019-01-01 07:20:00,20.0,100305,200974,오모떼 레이스 파운데이션 브라,속옷,59000,15689000.0
8,2019-01-01 07:40:00,20.0,100305,200974,오모떼 레이스 파운데이션 브라,속옷,59000,25370000.0
9,2019-01-01 08:00:00,20.0,100808,202377,CERINI by PAT 남성 소프트 기모 릴렉스팬츠,의류,59900,16133000.0


- 모두 잘 채워졌다.

## 3. 적절한 변수 변경하기

- 이제 연속형으로 표현되어 있는 mom_code(마더코드)와 pd_code(상품코드)를 살펴보자.
    - 숫자로 구성되어 있지만 연속형으로 사용하지 않을 것이기 때문에 문자열로 변경하자.
    - 이후 필요하다면 범주형 변수로 만들어주겠다.

In [20]:
df_mom_pd_code_cat = df_mom_code_ffill.astype({'mom_code':'str', 'pd_code':'str'})
df_mom_pd_code_cat.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 37372 entries, 0 to 38303
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   date      37372 non-null  datetime64[ns]
 1   exp_mins  37372 non-null  float64       
 2   mom_code  37372 non-null  object        
 3   pd_code   37372 non-null  object        
 4   pd_name   37372 non-null  object        
 5   pd_group  37372 non-null  object        
 6   pd_price  37372 non-null  int64         
 7   sales     37372 non-null  float64       
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 2.6+ MB


- 손쉽게 object로 변화된 걸 확인할 수 있다.
<br>
<br>
- 이제 date(방송일시) 칼럼을 활용하여 weekdays(요일)과 seasons(계절) 칼럼을 생성하겠다.
    - 이는 나중에 Seasonality(계절성)이 있는지 확인하는데 유용하게 할용될 것이다.
    - 우선 datetime 메소드를 활용하여 weekdays(요일) 변수를 만들어주겠다.

In [21]:
df_mom_pd_code_cat['weekdays'] = [date.strftime("%A") for date in df_mom_pd_code_cat['date']]
df_mom_pd_code_cat

,date,exp_mins,mom_code,pd_code,pd_name,pd_group,pd_price,sales,weekdays
0,2019-01-01 06:00:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,2099000.0,Tuesday
1,2019-01-01 06:00:00,20.0,100346,201079,테이트 여성 셀린니트3종,의류,39900,4371000.0,Tuesday
2,2019-01-01 06:20:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,3262000.0,Tuesday
3,2019-01-01 06:20:00,20.0,100346,201079,테이트 여성 셀린니트3종,의류,39900,6955000.0,Tuesday
4,2019-01-01 06:40:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,6672000.0,Tuesday
...,...,...,...,...,...,...,...,...,...
38299,2019-12-31 23:40:00,20.0,100448,201391,일시불쿠첸압력밥솥 6인용,주방,148000,10157000.0,Tuesday
38300,2020-01-01 00:00:00,20.0,100448,201383,무이자쿠첸압력밥솥 10인용,주방,178000,50929000.0,Wednesday
38301,2020-01-01 00:00:00,20.0,100448,201390,일시불쿠첸압력밥솥 10인용,주방,168000,104392000.0,Wednesday
38302,2020-01-01 00:00:00,20.0,100448,201384,무이자쿠첸압력밥솥 6인용,주방,158000,13765000.0,Wednesday


- 이제 계절변수를 만들어보자.
    1. winter
    2. spring
    3. summer
    4. autumn

In [22]:
df_mom_pd_code_cat['seasons'] = [str((int(month.strftime("%m"))%12+3//3)) for month in df_mom_pd_code_cat['date']]
df_mom_pd_code_cat

,date,exp_mins,mom_code,pd_code,pd_name,pd_group,pd_price,sales,weekdays,seasons
0,2019-01-01 06:00:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,2099000.0,Tuesday,2
1,2019-01-01 06:00:00,20.0,100346,201079,테이트 여성 셀린니트3종,의류,39900,4371000.0,Tuesday,2
2,2019-01-01 06:20:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,3262000.0,Tuesday,2
3,2019-01-01 06:20:00,20.0,100346,201079,테이트 여성 셀린니트3종,의류,39900,6955000.0,Tuesday,2
4,2019-01-01 06:40:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,6672000.0,Tuesday,2
...,...,...,...,...,...,...,...,...,...,...
38299,2019-12-31 23:40:00,20.0,100448,201391,일시불쿠첸압력밥솥 6인용,주방,148000,10157000.0,Tuesday,1
38300,2020-01-01 00:00:00,20.0,100448,201383,무이자쿠첸압력밥솥 10인용,주방,178000,50929000.0,Wednesday,2
38301,2020-01-01 00:00:00,20.0,100448,201390,일시불쿠첸압력밥솥 10인용,주방,168000,104392000.0,Wednesday,2
38302,2020-01-01 00:00:00,20.0,100448,201384,무이자쿠첸압력밥솥 6인용,주방,158000,13765000.0,Wednesday,2


- 잘 변화되었다.

- 이제 pd_group(상품군)을 분석해보자.
    - 총 몇 가지 상품군이 있는지 확인하자
    - 그리고 어떤 상품군이 존재하는지도 확인해보자.
    - 앞서 확인해 본 바로는 mom_code(마더코드)는 285개 이상 존재한다.
    - 정확한 숫자를 알기 위해 mom_code(마더코드)와 pd_code(상품코드)도 숫자를 확인해보자

In [23]:
len(set(df_mom_pd_code_cat['pd_group'].tolist()))

11

In [24]:
set(df_mom_pd_code_cat['pd_group'].tolist())

{'가구', '가전', '건강기능', '농수축', '생활용품', '속옷', '의류', '이미용', '잡화', '주방', '침구'}

In [25]:
len(set(df_mom_pd_code_cat['mom_code'].tolist())), len(set(df_mom_pd_code_cat['pd_code'].tolist()))

(687, 2042)

- 상품군을 총 11가지이다.
    - 마더코드는 총 687가지이며, 상품코드는 총 2042가지이다.
    - 분류를 위해서는 상품군만 활용하면 될 것 같다.
    - 다만 EDA작업을 위해서는 마더코드와 상품코드(혹은 상품명)을 활용하여 어떤 패턴이 존재하는지 알아보는 것이 유용할 것 같다.

In [26]:
df_mom_pd_code_cat.to_csv('df_data_revised.csv')

In [36]:
len(df_excel[df_excel['pd_name'].str.contains('마스크')])

53

In [37]:
df_excel[df_excel['pd_name'].str.contains('마스크')]

,date,exp_mins,mom_code,pd_code,pd_name,pd_group,pd_price,sales
1590,2019-01-18 10:00:00,20.000000,100227,200787,씨엔지코리아 방역용마스크 KF94 60매,생활용품,40900,13624000.0
1591,2019-01-18 10:20:00,20.000000,100227,200787,씨엔지코리아 방역용마스크 KF94 60매,생활용품,40900,22112000.0
1592,2019-01-18 10:40:00,20.000000,100227,200787,씨엔지코리아 방역용마스크 KF94 60매,생활용품,40900,22881000.0
2014,2019-01-22 13:00:00,20.000000,100227,200787,씨엔지코리아 방역용마스크 KF94 60매,생활용품,40900,8204000.0
2015,2019-01-22 13:20:00,20.000000,100227,200787,씨엔지코리아 방역용마스크 KF94 60매,생활용품,40900,15681000.0
2016,2019-01-22 13:40:00,20.000000,100227,200787,씨엔지코리아 방역용마스크 KF94 60매,생활용품,40900,16104000.0
2301,2019-01-25 14:00:00,20.000000,100798,202361,크린조이 황사 방역마스크 대형 105매,생활용품,59900,11222000.0
2302,2019-01-25 14:00:00,NaN,100798,202362,크린조이 황사 방역마스크 소형 105매,생활용품,59900,1284000.0
2303,2019-01-25 14:20:00,20.000000,100798,202361,크린조이 황사 방역마스크 대형 105매,생활용품,59900,22621000.0
2304,2019-01-25 14:20:00,NaN,100798,202362,크린조이 황사 방역마스크 소형 105매,생활용품,59900,4153000.0


In [38]:
df_excel[df_excel['mom_code'] == 100000]

,date,exp_mins,mom_code,pd_code,pd_name,pd_group,pd_price,sales
25275,2019-08-26 00:20:00,20.0,100000,200000,엘로엘 아쿠아클린 마스크,이미용,79900,12078000.0
25276,2019-08-26 00:40:00,20.0,100000,200000,엘로엘 아쿠아클린 마스크,이미용,79900,17338000.0
25277,2019-08-26 01:00:00,20.0,100000,200000,엘로엘 아쿠아클린 마스크,이미용,79900,18195000.0
25620,2019-08-30 01:00:00,20.0,100000,200000,엘로엘 아쿠아클린 마스크,이미용,79900,5905000.0
25621,2019-08-30 01:20:00,20.0,100000,200000,엘로엘 아쿠아클린 마스크,이미용,79900,11888000.0
25622,2019-08-30 01:40:00,15.0,100000,200000,엘로엘 아쿠아클린 마스크,이미용,79900,11158000.0


In [39]:
df_excel.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38309 entries, 0 to 38308
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   date      38309 non-null  datetime64[ns]
 1   exp_mins  21525 non-null  float64       
 2   mom_code  38309 non-null  int64         
 3   pd_code   38309 non-null  int64         
 4   pd_name   38309 non-null  object        
 5   pd_group  38309 non-null  object        
 6   pd_price  38309 non-null  int64         
 7   sales     37372 non-null  float64       
dtypes: datetime64[ns](1), float64(2), int64(3), object(2)
memory usage: 2.3+ MB


In [40]:
df_excel.describe()

,exp_mins,mom_code,pd_code,pd_price,sales
count,21525.000000,38309.000000,38309.000000,3.830900e+04,3.737200e+04
mean,20.174664,100390.972148,201219.923334,4.565537e+05,2.187305e+07
std,3.633757,249.927870,735.677344,7.260555e+05,2.019140e+07
min,2.466667,100000.000000,200000.000000,0.000000e+00,5.000000e+04
25%,20.000000,100155.000000,200550.000000,5.900000e+04,6.880750e+06
50%,20.000000,100346.000000,201167.000000,1.090000e+05,1.612950e+07
75%,20.000000,100596.000000,201863.000000,4.990000e+05,3.163125e+07
max,60.000000,100849.000000,202513.000000,7.930000e+06,3.220090e+08


In [43]:
df_exp_mins = df_excel[(df_excel['exp_mins'] != 20) & (~df_excel['exp_mins'].isnull())]
df_exp_mins

,date,exp_mins,mom_code,pd_code,pd_name,pd_group,pd_price,sales
23,2019-01-01 13:00:00,30.000000,100305,200970,오모떼 360도 텐션업 레이스 패키지 시즌4,속옷,109000,32273000.0
24,2019-01-01 13:30:00,30.000000,100305,200970,오모떼 360도 텐션업 레이스 패키지 시즌4,속옷,109000,79273000.0
25,2019-01-01 14:00:00,30.000000,100377,201220,그렉노먼 남성 구스다운 롱 벤치코트,의류,119000,11269000.0
27,2019-01-01 14:30:00,30.000000,100377,201220,그렉노먼 남성 구스다운 롱 벤치코트,의류,119000,40526000.0
53,2019-01-01 22:00:00,30.000000,100271,200896,헤스티지 엘레나 라쿤양가죽 콤비 롱코트,의류,299000,38067000.0
...,...,...,...,...,...,...,...,...
38160,2019-12-30 12:10:00,10.000000,100063,200153,대동모피 패션밍크 후드 롱코트 (19FW),의류,690000,37594000.0
38179,2019-12-30 18:20:00,60.000000,100072,200194,"삼성화재 천만안심 건강보험(암보험, 1907)",무형,0,NaN
38208,2019-12-31 01:40:00,17.166667,100439,201357,루시헨느 레이스 홀리데이 란쥬 패키지,속옷,89900,22453000.0
38243,2019-12-31 13:20:00,60.000000,100072,200194,"삼성화재 천만안심 건강보험(암보험, 1907)",무형,0,NaN


In [81]:
df_exp_mins_not20 = pd.DataFrame(df_exp_mins.sort_values(by='exp_mins').groupby('exp_mins')['date'].count())
df_exp_mins_not20.columns = ['date']
df_exp_mins_not20.reset_index(inplace=True)
df_exp_mins_not20

,exp_mins,date
0,2.466667,1
1,2.816667,1
2,3.000000,2
3,3.166667,1
4,3.283333,1
...,...,...
97,29.983333,1
98,30.000000,1064
99,30.100000,99
100,40.000000,1


In [91]:
df_exp_mins_not20[df_exp_mins_not20['exp_mins']%10 != 0]

,exp_mins,date
0,2.466667,1
1,2.816667,1
2,3.000000,2
3,3.166667,1
4,3.283333,1
...,...,...
94,26.333333,1
95,26.683333,2
96,27.083333,1
97,29.983333,1


In [93]:
df_exp_mins_not20[df_exp_mins_not20['exp_mins']%10 != 0]['date'].sum()

598

In [92]:
exp_mins_not20_lst = df_exp_mins_not20[df_exp_mins_not20['exp_mins']%10 != 0]['exp_mins'].tolist()
exp_mins_not20_lst

[2.466666666666667,
 2.816666666666667,
 3.0,
 3.1666666666666665,
 3.283333333333333,
 3.45,
 5.166666666666667,
 5.333333333333333,
 5.466666666666667,
 6.633333333333334,
 6.666666666666667,
 6.683333333333334,
 6.85,
 7.266666666666667,
 8.85,
 9.7,
 10.966666666666667,
 11.033333333333333,
 11.333333333333334,
 12.0,
 12.133333333333333,
 12.5,
 12.533333333333333,
 12.666666666666666,
 12.9,
 13.0,
 13.166666666666666,
 13.333333333333334,
 13.5,
 14.35,
 14.666666666666666,
 14.7,
 14.833333333333334,
 14.983333333333333,
 15.0,
 15.016666666666667,
 15.033333333333333,
 15.066666666666666,
 15.083333333333334,
 15.166666666666666,
 15.2,
 15.25,
 15.333333333333334,
 15.366666666666667,
 15.5,
 15.666666666666666,
 15.833333333333334,
 15.85,
 15.866666666666667,
 16.0,
 16.016666666666666,
 16.033333333333335,
 16.05,
 16.083333333333332,
 16.1,
 16.166666666666668,
 16.2,
 16.333333333333332,
 16.35,
 16.516666666666666,
 16.666666666666668,
 16.683333333333334,
 16.7,
 16.83

In [97]:
index_list = df_exp_mins[df_exp_mins['exp_mins'].isin(exp_mins_not20_lst)].index.tolist()

Int64Index([   66,   170,   265,   349,   446,   532,   621,   720,   800,
              895,
            ...
            37738, 37778, 37866, 37948, 37965, 37972, 38051, 38136, 38208,
            38308],
           dtype='int64', length=598)